In [1]:
import pandas as pd
import numpy as np

In [2]:
#import matplotlib.pyplot as plt 
#import matplotlib.colors as mcolors

In [3]:
data = pd.read_excel('manual.xlsx',sheet_name=0, index_col=0)
data = np.asarray(data, dtype=float)
print(data)

[[109.]
 [130.]
 [129.]
 [114.]
 [149.]
 [113.]
 [196.]
 [106.]
 [181.]
 [218.]
 [247.]
 [218.]
 [337.]
 [219.]
 [330.]
 [399.]
 [316.]
 [282.]
 [297.]
 [380.]
 [407.]
 [325.]
 [327.]
 [185.]
 [375.]
 [283.]
 [357.]
 [436.]
 [396.]
 [275.]
 [214.]
 [415.]
 [260.]
 [347.]]


In [4]:
#data_positif.plot( 'tanggal','jumlah_positif', title='Kasus Harian Positif COVID-19 di Indonesia',xlabel='bulan',ylabel='jumlah kasus',grid=True)

In [5]:
#data_sembuh.plot( 'tanggal' , 'jumlah_sembuh', color='green', title='Kasus Harian Sembuh COVID-19 di Indonesia',xlabel='bulan',ylabel='jumlah kasus',grid=True)

In [6]:
#data_meninggal.plot( 'tanggal' , 'jumlah_meninggal',color='red', title='Kasus Harian Meninggal COVID-19 di Indonesia',xlabel='bulan',ylabel='jumlah kasus',grid=True)

# Normalisasi Data

In [7]:
def normalisasi_minmax(data):
    normalisasiData = []
    maksimimal = np.max(data, axis=0)
    minimal = np.min(data, axis=0)
    for x in data:
        x_normalisasi = (x-maksimal)/(maksimimal-minimal)
        normalisasiData.append(round(float(x_normalisasi),5))
    return normalisasiData
normalisasiData = normalisasi(data)
datanormalisasi = pd.DataFrame(normalisasiData)

In [8]:
# pre-processing
def jumlahfitur(data, jumlah_fitur):
    input_fitur=list()
    target=list()
    for i in range(len(data)):
        index_terakhir= i+jumlah_fitur
        if index_terakhir>len(data)-1:
            break
        else:
            sequence_fitur = data[i:index_terakhir]
            sequence_target = data[index_terakhir]
            input_fitur.append(sequence_fitur)
            target.append(sequence_target)
    return input_fitur, target

In [9]:
jumlah_fitur = 2
hidden_neuron = 3

In [10]:
input_fitur, target=jumlahfitur(normalisasiData, jumlah_fitur)
dataFitur = pd.DataFrame(input_fitur,target)
dataFitur

,0,1
0.06970,0.00909,0.07273
0.02424,0.07273,0.06970
0.13030,0.06970,0.02424
0.02121,0.02424,0.13030
0.27273,0.13030,0.02121
0.00000,0.02121,0.27273
0.22727,0.27273,0.00000
0.33939,0.00000,0.22727
0.42727,0.22727,0.33939
0.33939,0.33939,0.42727


In [11]:
# pembagian dataset
# data training
data_train = input_fitur[0:int(len(input_fitur)*0.7)]
data_train = np.asarray(data_train)
target_train = target[0:int(len(target)*0.7)]
target_train = np.asarray(target_train)
# data testing
data_test = input_fitur[int(len(input_fitur)*0.7):]
data_test = np.asarray(data_test)
target_test = target[int(len(target)*0.7):]
target_test = np.asarray(target_test)
target_train = target_train.reshape(len(target_train),1)

In [12]:
len(data_train)

22

In [13]:
len(data_test)

10

# TRAINING

In [14]:
# TRAINING
# inisialiasi input weight
inputweight = pd.DataFrame(np.random.uniform(-1, 1,(hidden_neuron,jumlah_fitur)))
inputweight

,0,1
0,-0.502704,0.227121
1,-0.750233,0.095135
2,-0.094644,0.672179


In [15]:
inputweight.to_excel('manualisasi_inputweight.xlsx')

In [16]:
# inisialisasi bias
bias = pd.DataFrame(np.random.uniform(-.5, .5, hidden_neuron))
# bias

In [17]:
bias.to_excel('manualisasi_bias.xlsx')

In [18]:
def transpose (inputweight):
    transpose_inputweight = np.zeros((inputweight.shape[1],inputweight.shape[0]))
    for i in range(inputweight.shape[0]):
        for j in range(inputweight.shape[1]):
            transpose_inputweight[j][i] = inputweight.iloc[i,j]
    return transpose_inputweight
transpose_inputweight = transpose(inputweight)

In [19]:
inputWeight_transpose = transpose(inputweight)

In [20]:
bias_transpose = transpose(bias)
bias_transpose

array([[ 0.44577158, -0.28712195, -0.20090413]])

In [21]:
def perkalian_matriks(matriks1, matriks2):
    matriks = []
    for i in range(len(matriks1)):
        temp = []
        for j in range(len(matriks2[0])):
            value = 0
            for k in range(len(matriks2)):
                value += (matriks1[i][k]* matriks2[k][j])
            temp.append(value)
        matriks.append(temp)
    return np.asarray(matriks)

In [22]:
Hint = pd.perkalian_matriks(data_train,transpose_inputweight)+bias_transpose

AttributeError: module 'pandas' has no attribute 'perkalian_matriks'

In [ ]:
#Hint = np.add(XtrainWt,bias_transpose)
#Hint
# Hint

In [ ]:
# def tambahbias (XtrainWt,bias):
#     nilai_Hint = XtrainWt
#     for i in range (len(XtrainWt)):
#         for j in range(len(bias)):
#             nilai_Hint[i][j] = XtrainWt[i,j] + bias[j]
#     return nilai_Hint
# Hint = tambahbias(XtrainWt,bias)
# Hint

In [ ]:
# fungsi aktivasi
def hitung_h(Hint):
    h = (1/(1+np.exp(-1*Hint)))
    return h
h=hitung_h(Hint)
# h

In [ ]:
# transpose h
def h_transpose (h):
    transpose_h = np.zeros((h.shape[1],h.shape[0]))
    for i in range(h.shape[0]):
        for j in range(h.shape[1]):
            transpose_h[j][i] = h[i,j]
    return transpose_h
transpose_h = h_transpose(h)
# transpose_h

In [ ]:
# HT*H
perkalianHT_H = perkalian_matriks(transpose_h,h)

In [ ]:
def inverse(perkalianHT_H):
    inverseHT_H = []
    for i in range(len(perkalianHT_H)):
        temp_inverse=[]
        for j in range(2*len(perkalianHT_H[i])):
            if j < len(perkalianHT_H[i]):
                temp_inverse.append(perkalianHT_H[i][j])
            elif i == j-len(perkalianHT_H[i]):
                temp_inverse.append(1)
            else:
                temp_inverse.append(0)
        inverseHT_H.append(temp_inverse)
    for k in range(len(inverseHT_H)):
        key1 = inverseHT_H[k][k]
        for l in range(len(inverseHT_H[k])):
            inverseHT_H[k][l] /= key1
        for m in range(len(inverseHT_H)):
            if m == k:
                continue
            else:
                key2 = inverseHT_H[m][k]
                for n in range(len(inverseHT_H[k])):
                    inverseHT_H[m][n] -= (key2*inverseHT_H[k][n])
    for v in range (len(inverseHT_H)):
        for u in range(int(len(inverseHT_H[v])/2)):
            inverseHT_H[v].pop(0)
    return inverseHT_H
hasil_inverse = inverse(perkalianHT_H)

In [ ]:
hasilmoore = perkalian_matriks(hasil_inverse,transpose_h)
# hasilmoore

In [ ]:
# menghitung output weight
beta = perkalian_matriks(hasilmoore,target_train)
# beta

In [ ]:
# menghitung output layer
gama_train = perkalian_matriks (h,beta)
gama_train = abs(gama_train)
# gama_test

# DENORMALISASI

In [ ]:
# denormalisasi
def denormalisasi(data, hasil):
    denormalisasiData = []
    maximum = np.max(data, axis=0)
    minimum = np.min(data, axis=0)
    for x in hasil:
        x_denormalisasi = x*(maximum-minimum)+minimum
        denormalisasiData.append(round(float(x_denormalisasi)))
    return denormalisasiData

In [ ]:
hasil_denormalisasi_train = denormalisasi(data, gama_train)

In [ ]:
targetError = denormalisasi(data,target_train)

# PERHITUNGAN METRIC EVALUASI

In [ ]:
"""def evaluasi(test, hasil_peramalan):
    temp = 0
    for i,j in zip(test,hasil_peramalan):
        print("hasil peramalan = {} target = {}".format(j,i))
        temp += (abs(i-j)/(j))*100
    MAPE = temp/(len(hasil_peramalan))
    print("MAPE =",MAPE)"""

In [ ]:
def smape(test, hasil_peramalan):
    temp = 0
    for i,j in zip(test,hasil_peramalan):
        #print("target = {} hasil peramalan = {} ".format(i,j))
        temp += 2*(abs(i-j)/(i+j))
    smape = temp/len(hasil_peramalan)
    print("sMAPE =",smape)

In [ ]:
#cek = smape(targetError, hasil_denormalisasi_train)

In [ ]:
print("fitur =",jumlah_fitur)
print("hidden neuron =",hidden_neuron)

In [ ]:
train = smape(targetError,hasil_denormalisasi_train)

In [ ]:
#mape_train = np.sum(np.absolute((error*100)/len(targetError)))
#mape_train 

# TESTING

In [ ]:
# TESTING
Hint_test = perkalian_matriks(data_test, transpose_inputweight)+bias_transpose
Hint_test
# XtestWt

In [ ]:
"""def tambahbias_testing (XtestWt,bias):
    nilai_HintTest = XtestWt
    for i in range (len(XtestWt)):
        for j in range(len(bias)):
            nilai_HintTest[i][j] = XtestWt[i,j] + bias[j]
    return nilai_HintTest
Hint_test = tambahbias_testing(XtestWt,bias)
# Hint_test"""

In [ ]:
# menghitung fungsi aktivasi
h_test=hitung_h(Hint_test)
h_test

In [ ]:
beta

In [ ]:
# menghitung output layer
gama_test = perkalian_matriks (h_test,beta)
gama_test = abs(gama_test)
# gama_test

In [ ]:
target_test

In [ ]:
hasil_denormalisasi_test = denormalisasi(data, gama_test)

In [ ]:
targetError_test = denormalisasi(data,target_test)

In [ ]:
print("fitur =",jumlah_fitur)
print("hidden neuron =",hidden_neuron)

In [ ]:
test = smape(targetError_test,hasil_denormalisasi_test)

In [ ]:
"""plt.plot(targetError)
plt.plot(hasil_denormalisasi_train)
plt.legend(['Train Data', 'ELM Predictions'])"""

In [ ]:
"""plt.plot(targetError_test)
plt.plot(hasil_denormalisasi_test)
plt.legend(['Test Data', 'ELM Predictions'])"""